# CS229: Problem Set 4
## Problem 4: Independent Component Analysis


**C. Combier**

This iPython Notebook provides solutions to Stanford's CS229 (Machine Learning, Fall 2017) graduate course problem set 3, taught by Andrew Ng.

The problem set can be found here: [./ps4.pdf](ps4.pdf)

I chose to write the solutions to the coding questions in Python, whereas the Stanford class is taught with Matlab/Octave.

## Notation

- $x_i$ is the $i^{th}$ feature vector
- $y_i$ is the expected outcome for the $i^{th}$ training example
- $z_i$'s are the latent (hidden) variables
- $m$ is the number of training examples
- $n$ is the number of features

For clarity, I've inlined the code of the provided helper function ```belsej.py```.

## Dependencies

I installed ```sounddevice``` to Anaconda with the following command:

```conda install -c conda-forge python-sounddevice ```

First, let's set up the environment and write helper functions:

- ```normalize``` ensures all mixes have the same volume
- ```load_data``` loads the mix
- ```play``` plays the audio using ```sounddevice```

In [2]:
### Independent Components Analysis
###
### This program requires a working installation of:
###
### On Mac:
### conda install -c conda-forge python-sounddevice
###

import sounddevice as sd
import numpy as np

Fs = 11025

def normalize(dat):
    return 0.99 * dat / np.max(np.abs(dat))

def load_data():
    mix = np.loadtxt('data/mix.dat')
    return mix

def play(vec):
    sd.play(vec, Fs, blocking=True)

Next we write a numerically stable sigmoid function, to avoid overflows:

In [1]:
# Numerically stable sigmoid
def sigmoid(x):
    return np.where(x >= 0, 1 / (1 + np.exp(-x)), np.exp(x) / (1 + np.exp(x)))

The following functions calculates the weights to separate the independent components of the five mixes, using stochastic gradient descent and annealing to speed up convergence.

In [3]:
def unmixer(X):
    M, N = X.shape
    W = np.eye(N)

    anneal = [0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.02, 0.02, 0.01, 0.01,
              0.005, 0.005, 0.002, 0.002, 0.001, 0.001]
    print('Separating tracks ...')
    for alpha in anneal:
        for xi in X:
            W += alpha * (np.outer(1 - 2 * sigmoid(np.dot(W, xi.T)), xi) + np.linalg.inv(W.T))
    return W

Finally, this last function unmixes the 5 mixes to extract the independent components.

In [4]:
def unmix(X, W):
    S = np.zeros(X.shape)
    S = X.dot(W.T)
    return S

Now, we load the mix data:

In [8]:
X = normalize(load_data())
for i in range(X.shape[1]):
    print('Playing mixed track %d' % i)
    play(X[:, i])

Playing mixed track 0
Playing mixed track 1
Playing mixed track 2
Playing mixed track 3
Playing mixed track 4


Next, we run Independent Component Analysis and separate the components in the mix:

In [7]:
W = unmixer(X)
S = normalize(unmix(X, W))

Separating tracks ...


Finally, we play the separated components:

In [9]:
for i in range(S.shape[1]):
    print('Playing separated track %d' % i)
    play(S[:, i])

Playing separated track 0
Playing separated track 1
Playing separated track 2
Playing separated track 3
Playing separated track 4
